In [1]:
import time
import numpy as np
import tensorflow as tf
import random

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
np.random.seed(1)


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
from os import listdir
import os
print os.getcwd()
listdir("/content/gdrive/My Drive/Colab Notebooks")

/content


['BC_USDT_30min',
 'BTC_USD_TPU.ipynb',
 'BTC_USD_TPU_v2.ipynb',
 'glove.6B.50d.txt',
 'text8_processed',
 'LanguageModeling_v1.ipynb']

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map



In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("/content/gdrive/My Drive/Colab Notebooks/glove.6B.50d.txt")

In [0]:
import re
from collections import Counter    

def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' . <SPLIT>')
    #text = text.replace(',', ' <COMMA> ')
    #text = text.replace('"', ' <QUOTATION_MARK> ')
    #text = text.replace(';', ' <SEMICOLON> ')
    #text = text.replace('!', ' <EXCLAMATION_MARK> ')
    #text = text.replace('?', ' <QUESTION_MARK> ')
    #text = text.replace('(', ' <LEFT_PAREN> ')
    #text = text.replace(')', ' <RIGHT_PAREN> ')
    #text = text.replace('--', ' <HYPHENS> ')
    #text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    #text = text.replace(':', ' <COLON> ')
    sentences = text.split('<SPLIT>')
    
    words = [sentence.split() for sentence in sentences]
    words2 = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words2)
    #trimmed_words = [[x for x in y if word_counts[x] > 5] for y in words]
    words_not_in_glove = set(word_counts.keys()) - set(word_to_index.keys())
    
    trimmed_words = [[x for x in y if ((x not in words_not_in_glove) and ( word_counts[x] > 50)) ] for y in words]
    return trimmed_words
    

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return list(zip(x_batches, y_batches))


def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

def tokenize_list(l_words, vocab_to_int):
    tokenized_list = [vocab_to_int[word] for word in l_words]
    return tokenized_list
    
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [7]:
file1 = open("/content/gdrive/My Drive/Colab Notebooks/text8_processed","r+") 
text =  file1.read() 
print text[:1000]
words = preprocess(text)
print words[0:4]

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution. whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists. the word anarchism is derived from the greek without archons ruler chief king . anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means. anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state. the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society. in place of what are regarded as authoritarian political structures and coercive economic in

In [8]:
len(words)

577782

In [0]:
def training_data(data, length = 5):
    input_sequences = []
    for line in data:
        for i in range(1, len(line)):
            l = random.randint(1,length)
            n_gram_sequence = line[max(0, (i-l)):i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

In [0]:
train_data = training_data(words)

In [0]:
#train_data = train_data[:10000]

In [12]:
print len(train_data)
print train_data[:10]

11652975
[['anarchism', 'originated'], ['anarchism', 'originated', 'as'], ['anarchism', 'originated', 'as', 'a'], ['a', 'term'], ['anarchism', 'originated', 'as', 'a', 'term', 'of'], ['a', 'term', 'of', 'abuse'], ['as', 'a', 'term', 'of', 'abuse', 'first'], ['of', 'abuse', 'first', 'used'], ['first', 'used', 'against'], ['first', 'used', 'against', 'early']]


In [13]:
X = [l[:-1] for l in train_data]
print "Done with X"
Y = [l[-1] for l in train_data]
starting_word = [l[0] for l in train_data]

print len(X)
print len(Y)

Done with X
11652975
11652975


In [14]:
print len(set(Y))

print len(starting_word)
unique_words = set(starting_word + Y)
print len(unique_words)

15058
11652975
15058


In [0]:
vocab_to_int, int_to_vocab = create_lookup_tables(list(unique_words))
Y_classes = len(int_to_vocab.keys())
Y_tokenized = tokenize_list(Y, vocab_to_int)

In [16]:
len(Y_tokenized)

11652975

In [17]:
maxLen = len(max(X, key=len))
maxLen

5

In [0]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = len(X)                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i]
        
        # Initialize j to 0
        j = 0
        l_sentence = len(X[i])
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, (max_len - l_sentence + j)] = word_to_index[w]
            # Increment j to j + 1
            j = j + 1
            
    ### END CODE HERE ###
    
    return X_indices

In [0]:
Y_tokenized_array = np.array(Y_tokenized)
X_train_indices = sentences_to_indices(X, vocab_to_int, maxLen)

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)
    
    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
        
    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    #embedding_layer = tf.keras.layers.Embedding(vocab_len,emb_dim, weights = [emb_matrix], trainable = False)
    ### END CODE HERE ###
    
    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    #embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    #embedding_layer.set_weights([emb_matrix])
    
    return vocab_len, emb_dim, emb_matrix
    #return embedding_layer

In [0]:
vocab_len, emb_dim, emb_matrix = pretrained_embedding_layer(word_to_vec_map, vocab_to_int)
#el= pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [0]:
def Language_Modeling_v2(input_shape2,word_to_vec_map, word_to_index):
    model = tf.keras.models.Sequential()
    #model.add(Input(shape = input_shape, dtype='float32'))
    #model.add(el)
    model.add(tf.keras.layers.Embedding(vocab_len,emb_dim, weights = [emb_matrix], trainable = False,input_shape=input_shape2))
    #model.add(tf.keras.layers.Embedding(vocab_len,emb_dim, weights = [emb_matrix], trainable = False,input_shape=input_shape2).build((None,)))
    #model.add(tf.keras.layers.Embedding(vocab_len,emb_dim, trainable = False,input_shape=input_shape2).build((None,)).set_weights([emb_matrix]))
    
    model.add(tf.keras.layers.LSTM(128,return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.LSTM(128,return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(Y_classes))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

In [23]:
maxLen = len(max(X, key=len))
maxLen

5

In [24]:
model = Language_Modeling_v2((maxLen,),word_to_vec_map, vocab_to_int )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 50)             752950    
_________________________________________________________________
lstm (LSTM)                  (None, 5, 128)            91648     
_________________________________________________________________
dropout (Dropout)            (None, 5, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 15058)             1942482   
_________________________________________________________________
activation (Activation)      (None, 15058)             0         
Total para

In [25]:
import os
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

INFO:tensorflow:Querying Tensorflow master (grpc://10.1.209.162:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8100473102259489895)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14916173854151612728)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 17848467561482493875)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9209190503702516931)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2160008521271984027)
INFO:tensorflow:*** Available Device: _DeviceAtt

In [0]:
tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['categorical_accuracy']
)

In [0]:
def train_gen(batch_size):
  while True:
    offset = np.random.randint(0, X_train_indices.shape[0] - batch_size)
    #yield X_train_indices[offset:offset+batch_size], Y_train[offset:offset + batch_size]
    yield X_train_indices[offset:offset+batch_size], convert_to_one_hot(Y_tokenized_array[offset:offset + batch_size], C = Y_classes)

In [28]:
import time
start = time.time()

tpu_model.fit_generator(
    train_gen(128),
    epochs=1,
    steps_per_epoch=10000,
    #validation_data=(X_test, Y_test),
    #class_weight = {0:1,1:8,2:8,3:40}
)

end = time.time()
print(end - start)

Epoch 1/1
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name=u'core_id0'), TensorSpec(shape=(16, 5), dtype=tf.float32, name=u'embedding_input_10'), TensorSpec(shape=(16, 15058), dtype=tf.float32, name=u'activation_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for embedding_input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 4.54245901108 secs
INFO:tensorflow:Setting weights on TPU model.
10000/10000 [==============================] - 5243s 524ms/step - loss: 6.8655 - categorical_accuracy: 0.0675
5243.60794783


In [0]:
#cpu_model = tpu_model.sync_to_cpu()
#cpu_model.save('/content/gdrive/My Drive/Colab Notebooks/language_model_v1.h5')

In [30]:
cpu_model = tpu_model.sync_to_cpu()
cpu_model.save('/content/gdrive/My Drive/Colab Notebooks/language_model_v2.h5')

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
def string_to_input(string,word_to_index, max_len ):
  sentence = string.strip().lower().split()
  sentence2 = [word for word in sentence if word in word_to_index.keys()]
  if len(sentence2) > 5:
    sentence2 = sentence2[-5:]
  return sentences_to_indices([sentence2], word_to_index, max_len)

In [0]:
pred = cpu_model.predict(string_to_input("This is a bat", vocab_to_int, 5))

In [36]:
np.argmax(pred)

13008

In [37]:
int_to_vocab[np.argmax(pred)]

'of'

In [0]:
string = "police"

In [122]:
print string_to_input(string, vocab_to_int, 5)
int_to_vocab[np.argmax(cpu_model.predict(string_to_input(string, vocab_to_int, 5)))]

[[   0.    0.    0.    0. 8593.]]


'the'